In [2]:
!pip install selenium

     |████████████████████████████████| 904 kB 20 kB/s eta 0:00:011
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [8]:
!pip install tensorflow

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install google.colab

     |████████████████████████████████| 72 kB 196 kB/s eta 0:00:011
     |████████████████████████████████| 64 kB 234 kB/s eta 0:00:011
     |████████████████████████████████| 104 kB 217 kB/s eta 0:00:01
     |████████████████████████████████| 758 kB 23 kB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 17 kB/s eta 0:00:013
     |██████████████████▏             | 9.0 MB 12 kB/s eta 0:09:0949     |█████                           | 2.5 MB 17 kB/s eta 0:13:09

In [9]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

ModuleNotFoundError: No module named 'google.colab'

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


In [6]:
def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

In [7]:
gpu()

NameError: name 'tf' is not defined

In [3]:
 # -*- coding: utf-8 -*-
# 使用Selenium, 北京安居客->小区->二手房信息
import json
import requests
from lxml import etree
import time
from selenium import webdriver
import pandas as pd 

# 需要将chromedriver放到Chrome\Application目录下
driver = webdriver.Chrome()

# 去空格，去换行\n
def format_str(str):
    return str.replace('\n', '').replace(' ', '')

# 对页面进行抓取分析
def work(request_url):
    driver.get(request_url)
    time.sleep(1)
    html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
    html = etree.HTML(html)
    # 设置需要抓取字段的xpath
    titles = html.xpath("/html/body/div[@class='w1180']/div[@class='maincontent']/div[@id='list-content']/div[@class='li-itemmod']/div[@class='li-info']//a/@title")
    links = html.xpath("/html/body/div[@class='w1180']/div[@class='maincontent']/div[@id='list-content']/div[@class='li-itemmod']/div[@class='li-info']//a/@href")
    addresses = html.xpath("/html/body/div[@class='w1180']/div[@class='maincontent']/div[@id='list-content']/div[@class='li-itemmod']/div[@class='li-info']/address/text()")
    prices = html.xpath("/html/body/div[@class='w1180']/div[@class='maincontent']/div[@id='list-content']/div[@class='li-itemmod']/div[@class='li-side']/p[1]/strong/text()")
    to_last_months = html.xpath("/html/body/div[@class='w1180']/div[@class='maincontent']/div[@id='list-content']/div[@class='li-itemmod']/div[@class='li-side']/p[2]/text()")
    completion_dates = html.xpath("/html/body/div[@class='w1180']/div[@class='maincontent']/div[@id='list-content']/div[@class='li-itemmod']/div[@class='li-info']/p[1]/text()[1]")
    houses = pd.DataFrame(columns = ['title', 'link', 'address', 'price', 'to_last_month', 'completion_date'])
    for i in range(len(titles)):
        # 设置抓取的房源
        temp = {}
        temp['title'] = format_str(titles[i])
        temp['link'] = format_str(links[i])
        temp['address'] = format_str(addresses[i])
        temp['price'] = format_str(prices[i])
        temp['to_last_month'] = format_str(to_last_months[i])
        temp['completion_date'] = format_str(completion_dates[i])
        # 添加房源
        houses = houses.append(temp,ignore_index=True)
    return houses

# 抓取10页北京房价数据
page_num = 10
base_url = 'https://beijing.anjuke.com/community/p'
houses = pd.DataFrame(columns = ['title', 'link', 'address', 'price', 'to_last_month', 'completion_date'])
for i in range(page_num):
    request_url = base_url+str(i+1)
    print(request_url)
    # 抓取该页的房源信息
    temp = work(request_url)
    houses = houses.append(temp)
    print(temp)
houses.to_csv('house_prices_bj.csv')




WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
